In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import statsmodels.api as sm
import dash
from dash import dcc, html, Input, Output


In [2]:
# 4.2.e) Find a geojson (or shapefile or else) of your choice online, import it and plot it with the data of your choice (found online or made up).
regions = ["Piemonte", "Valle d'Aosta", "Lombardia", "Trentino Alto Adige", "Veneto",
           "Friuli Venezia Giulia", "Liguria", "Emilia-Romagna", "Toscana", "Umbria",
           "Marche", "Lazio", "Abruzzo", "Molise", "Campania", "Puglia", "Basilicata",
           "Calabria", "Sicilia", "Sardegna"]

geojson_path = 'https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_regions.geojson'

## Random values
# values = np.random.randint(0, 5, len(regions)) 
# values_type = 'Random values'
## Number of football clubs in 1st division
values = [2, 0, 4, 0, 1, 1, 2, 4, 1, 0, 0, 2, 0, 0, 1, 1, 0, 0, 0, 1]
values_type = 'Number of football clubs in 1st division (2020)'
## GDP per capita
# values = [30300, 35200, 38200, 39074, 33100, 31000, 29678, 35300, 30500, 24300, 26600, 32900, 24400, 19500, 18200, 18000, 20800, 17100, 17400, 20300]
# values_type = 'GDP per capita (2020)'

# Create the df
df = pd.DataFrame({'regions': regions,'values': values}, columns = ['regions', 'values'])

# Make the figure
fig = px.choropleth(df, geojson=geojson_path, 
                locations='regions', 
                color='values', 
                color_continuous_scale='viridis', 
                featureidkey='properties.reg_name', # property in the GeoJSON file that corresponds to the regions' identifiers
                )
fig.update_layout(title_text=f'{values_type}', title_x=0.5,margin={"r": 0, "t": 40, "l": 0, "b": 0})
fig.update_geos(fitbounds="locations", visible=False)

fig.show()


# With go.Choropleth instead of px.Choropleth

In [3]:
# With go.Choropleth instead of px.Choropleth
fig = go.Figure(go.Choropleth(
    geojson=geojson_path,
    locations=df['regions'],
    z=df['values'],
    colorscale='Viridis',
    colorbar=dict(
        title='Values'
    ),
    featureidkey='properties.reg_name',
))

fig.update_layout(
    geo=dict(
        fitbounds="locations",
        visible=False
    )
)

fig.show()

# Add a button

In [4]:
# Add a button

# Number of football clubs in 1st division (2020)
values1 = [2, 0, 4, 0, 1, 1, 2, 4, 1, 0, 0, 2, 0, 0, 1, 1, 0, 0, 0, 1]

# GDP per capita (2020)
values2 = [30300, 35200, 38200, 39074, 33100, 31000, 29678, 35300, 30500, 24300, 26600, 32900, 24400, 19500, 18200, 18000, 20800, 17100, 17400, 20300]

df = pd.DataFrame({'regions': regions, 'values1': values1, 'values2': values2}, columns=['regions', 'values1', 'values2'])

fig = px.choropleth(df, geojson=geojson_path,
                    locations='regions',
                    color='values1',
                    color_continuous_scale='viridis',
                    featureidkey='properties.reg_name'
                    )
fig.update_geos(fitbounds="locations", visible=False)

buttons = [
    dict(label="Number of Football Clubs",
         method="update",
         args=[{"z": [df['values1']],
                "colorbar.title.text": "Number of Clubs",
                "colorscale": 'viridis',
                "colorbar.tickvals": [0, max(df['values1']) / 2, max(df['values1'])]},
                {"title":'Number of Football Clubs'}                    
                ]),
    dict(label="GDP per Capita",
         method="update",
         args=[{"z": [df['values2']],
                "colorbar.title.text": "GDP per Capita",
                "colorscale": 'viridis',
                "colorbar.tickvals": [0, max(df['values2']) / 2, max(df['values2'])]},
                {"title":'GDP per Capita'}                    
                ]),
]

fig.update_layout(updatemenus=[dict(type="buttons",
                                    direction="right",
                                    x=0.8,
                                    y=1.15,
                                    showactive=True,
                                    buttons=buttons)],
                                    title='Number of Football Clubs',
                                    title_x=0.3, # Move the title 
                                    coloraxis_colorbar=dict(x=0.8) # Move the colorbar
                                    )

fig.show()

# 2 subplots of choropleth

In [5]:
# 2 subplots of choropleth

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "choropleth"}, {"type": "choropleth"}]]
)

fig.add_trace(
    go.Choropleth(
        geojson=geojson_path,
        locations=df['regions'],
        z=df['values1'],
        colorscale='Viridis',
        colorbar=dict(
            title='Number of clubs'
        ),
        colorbar_x=0.4,
        featureidkey='properties.reg_name',
    ),
    row=1, col=1
)

fig.add_trace(
    go.Choropleth(
        geojson=geojson_path,
        locations=df['regions'],
        z=df['values2'],
        colorscale='Viridis',
        colorbar=dict(
            title='GdP'
        ),
        featureidkey='properties.reg_name',
    ),
    row=1, col=2
)

fig.update_geos(
    fitbounds="locations",
    visible=False,
)
fig.update_layout(
    title='Number of Clubs in 1st Division and GdP in Italy', title_x=0.5
)
fig.show()

# 3 subplots 

In [6]:
# 3 subplots 

# Calculate regression line
X = df['values1']
Y = df['values2']
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
predicted_values = model.predict(X)

# Assuming you have your data loaded in the DataFrame df
# Calculate correlation coefficient
correlation_coefficient = df['values1'].corr(df['values2'])
# 0.5, moderate correlation

# Create a new subplot for correlation heatmap
fig = make_subplots(
    rows=1, cols=3,
    specs=[[{"type": "choropleth"}, {"type": "choropleth"}, {"type": "heatmap"}]]
)

# Add the first choropleth plot
fig.add_trace(
    go.Choropleth(
        geojson=geojson_path,
        locations=df['regions'],
        z=df['values1'],
        colorscale='Viridis',
        colorbar=dict(
            title='Number of clubs'
        ),
        colorbar_x=0.,
        featureidkey='properties.reg_name',
    ),
    row=1, col=1
)

# Add the second choropleth plot
fig.add_trace(
    go.Choropleth(
        geojson=geojson_path,
        locations=df['regions'],
        z=df['values2'],
        colorscale='Viridis',
        colorbar=dict(
            title='GdP'
        ),
        colorbar_x=0.35,
        featureidkey='properties.reg_name',
    ),
    row=1, col=2
)

# Add the scatter plot 
fig.add_trace(
    go.Scatter(
        x=df['values1'],
        y=df['values2'],
        mode='markers',
        marker=dict(
            color='blue',
            size=8,
            opacity=0.5,
        ),
        name='Region'  # Change the legend name here
    ),
    row=1, col=3
)
# Add the regression line
fig.add_trace(
    go.Scatter(
        x=df['values1'],
        y=predicted_values,
        mode='lines',
        marker=dict(color='red'),
        name='Regression Line'
    ),
    row=1, col=3
)
fig.update_layout(
    xaxis_title="Number of Clubs",
    yaxis_title="GdP"
)
# Add correlation coefficient as annotation
fig.add_annotation(
    x=0.9,
    y=0.9,
    xref="paper",
    yref="paper",
    text=f"Pearson corr: {correlation_coefficient:.2f}",
    showarrow=False,
    font=dict(
        size=12,
        color="red"
    ),
)

fig.update_geos(
    fitbounds="locations",
    visible=False,
)
fig.update_layout(
    title='Number of Clubs in 1st Division and GdP in Italy', title_x=0.5
)
fig.show()

# Button example, with Dash

In [7]:
# Button example, with Dash

df = pd.DataFrame({'regions': regions, 'values1': values1, 'values2': values2}, columns=['regions', 'values1', 'values2'])

fig = px.choropleth(df, geojson=geojson_path,
                    locations='regions',
                    color='values1',
                    color_continuous_scale='viridis',
                    featureidkey='properties.reg_name'
                    )
fig.update_geos(fitbounds="locations", visible=False)

buttons = [
    dict(label="Number of Football Clubs",
         method="update",
         args=[{"z": [df['values1']],
                "colorbar.title.text": "Number of Clubs",
                "colorscale": 'viridis',
                "colorbar.tickvals": [0, max(df['values1']) / 2, max(df['values1'])]},
                {"title":'Number of Football Clubs'}                    
                ]),
    dict(label="GDP per Capita",
         method="update",
         args=[{"z": [df['values2']],
                "colorbar.title.text": "GDP per Capita",
                "colorscale": 'viridis',
                "colorbar.tickvals": [0, max(df['values2']) / 2, max(df['values2'])]},
                {"title":'GDP per Capita'}                    
                ]),
]

fig.update_layout(updatemenus=[dict(type="buttons",
                                    direction="right",
                                    x=0.8,
                                    y=1.15,
                                    showactive=True,
                                    buttons=buttons)],
                                    title='Number of Football Clubs',
                                    title_x=0.3, # Move the title 
                                    coloraxis_colorbar=dict(x=0.8) # Move the colorbar
                                    )

app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div(children='App with Italian values'),   
    dcc.Graph(id='choropleth-map', figure=fig)
])

if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


# Same but using callback and dash buttons instead of px buttons

In [ ]:
df = pd.DataFrame({'regions': regions, 'values1': values1, 'values2': values2},
                  columns=['regions', 'values1', 'values2'])

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.RadioItems(options=['Football Clubs', 'GdP'], value='values1', id='controls-and-radio-item'),
    dcc.Graph(id='choropleth-map', figure={})
])

@app.callback(
   Output(component_id='choropleth-map', component_property='figure'),
   Input(component_id='controls-and-radio-item', component_property='value')
)
def update_graph(value_type):
    # Update the figure properties instead of creating a new dcc.Graph
    if value_type == 'Football Clubs':
        title = 'Number of Football Clubs'
        values = 'values1'
    else:
        title = 'GDP per Capita'
        values = 'values2'
    fig = px.choropleth(df, geojson=geojson_path,
                    locations='regions',
                    color=values,
                    color_continuous_scale='viridis',
                    featureidkey='properties.reg_name',
                    title=title
                    )
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(title_x=0.5)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external")